In [95]:
import pandas as pd
df = pd.read_csv('Supervised_Learning_Dataset_2.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1264959 entries, 0 to 1264958
Data columns (total 17 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   CRASH DATE                 1264959 non-null  object 
 1   CRASH TIME                 1264959 non-null  object 
 2   CRASH DAY                  1264959 non-null  object 
 3   LATITUDE                   1264959 non-null  float64
 4   LONGITUDE                  1264959 non-null  float64
 5   holiday_name               32052 non-null    object 
 6   is_holiday                 1264959 non-null  bool   
 7   NUMBER OF PERSONS INJURED  1264959 non-null  float64
 8   NUMBER OF PERSONS KILLED   1264959 non-null  float64
 9   num_vehicles_involved      1264959 non-null  int64  
 10  avg_temp                   1264959 non-null  float64
 11  snow_depth                 1264709 non-null  float64
 12  precip                     1264959 non-null  float64
 13  max_wind_spd

In [96]:
df.drop(columns=['NUMBER OF PERSONS INJURED', 'num_vehicles_involved'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1264959 entries, 0 to 1264958
Data columns (total 15 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   CRASH DATE                1264959 non-null  object 
 1   CRASH TIME                1264959 non-null  object 
 2   CRASH DAY                 1264959 non-null  object 
 3   LATITUDE                  1264959 non-null  float64
 4   LONGITUDE                 1264959 non-null  float64
 5   holiday_name              32052 non-null    object 
 6   is_holiday                1264959 non-null  bool   
 7   NUMBER OF PERSONS KILLED  1264959 non-null  float64
 8   avg_temp                  1264959 non-null  float64
 9   snow_depth                1264709 non-null  float64
 10  precip                    1264959 non-null  float64
 11  max_wind_spd              1264959 non-null  float64
 12  percent_licensed          1233534 non-null  float64
 13  percent_unlicensed        1

In [97]:
df['is_holiday'] = df["is_holiday"].astype(int)
df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'], format='%Y-%m-%d')
df['CRASH TIME'] = pd.to_datetime(df['CRASH TIME'], format='%H:%M').dt.time
df['day_of_year'] = df['CRASH DATE'].dt.dayofyear
df['time_of_day'] = df['CRASH TIME'].apply(lambda time: time.hour * 60 + time.minute)
df.drop(columns=['CRASH DATE', 'CRASH TIME'], inplace=True)
day_num = {
    'Sunday': 0,
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6
}
df['CRASH DAY'] = df['CRASH DAY'].map(day_num)
df = pd.get_dummies(df, columns = ['holiday_name'])
df.iloc[:, 14:31] = df.iloc[:, 14:31].astype(int)
df['percent_unknown'] = 0
df['percent_unknown'] = df[['percent_licensed', 'percent_unlicensed', 'percent_permit']].isnull().all(axis=1) * 100
df[['percent_licensed', 'percent_unlicensed', 'percent_permit']] = df[['percent_licensed', 'percent_unlicensed', 'percent_permit']].fillna(0)
df.dropna(subset=['snow_depth'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1264709 entries, 0 to 1264958
Data columns (total 31 columns):
 #   Column                                                        Non-Null Count    Dtype  
---  ------                                                        --------------    -----  
 0   CRASH DAY                                                     1264709 non-null  int64  
 1   LATITUDE                                                      1264709 non-null  float64
 2   LONGITUDE                                                     1264709 non-null  float64
 3   is_holiday                                                    1264709 non-null  int64  
 4   NUMBER OF PERSONS KILLED                                      1264709 non-null  float64
 5   avg_temp                                                      1264709 non-null  float64
 6   snow_depth                                                    1264709 non-null  float64
 7   precip                                            

In [98]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

#scale numeric data such that features with larger scales do not disproportionately impact models
X = df.drop(columns=['NUMBER OF PERSONS KILLED'])
y = df['NUMBER OF PERSONS KILLED']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

#split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [99]:
"""
#decision tree with 1 level for baseline oneR
reg = DecisionTreeRegressor(random_state=42, max_depth=1)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("Decision Tree Regression MSE:", mse)
print("Decision Tree Regression RMSE:", rmse)
"""

Decision Tree Regression MSE: 0.002137167150928095
Decision Tree Regression RMSE: 0.046229505198823996


In [100]:
"""
#linear regression
reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("Linear Regression MSE:", mse)
print("Linear Regression RMSE:", rmse)
"""

'\n#linear regression\nreg = LinearRegression()\nreg.fit(X_train, y_train)\ny_pred = reg.predict(X_test)\n\nmse = mean_squared_error(y_test, y_pred)\nrmse = np.sqrt(mse)\nprint("Linear Regression MSE:", mse)\nprint("Linear Regression RMSE:", rmse)\n'

In [101]:
"""
#knn
reg = KNeighborsRegressor(n_neighbors=5)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("K-Nearest Neighbors Regression MSE:", mse)
print("K-Nearest Neighbors Regression RMSE:", rmse)
"""

'\n#knn\nreg = KNeighborsRegressor(n_neighbors=5)\nreg.fit(X_train, y_train)\ny_pred = reg.predict(X_test)\n\nmse = mean_squared_error(y_test, y_pred)\nrmse = np.sqrt(mse)\nprint("K-Nearest Neighbors Regression MSE:", mse)\nprint("K-Nearest Neighbors Regression RMSE:", rmse)\n'

In [102]:
"""
#decision tree
reg = DecisionTreeRegressor(random_state=42)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("Decision Tree Regression MSE:", mse)
print("Decision Tree Regression RMSE:", rmse)
"""

'\n#decision tree\nreg = DecisionTreeRegressor(random_state=42)\nreg.fit(X_train, y_train)\ny_pred = reg.predict(X_test)\n\nmse = mean_squared_error(y_test, y_pred)\nrmse = np.sqrt(mse)\nprint("Decision Tree Regression MSE:", mse)\nprint("Decision Tree Regression RMSE:", rmse)\n'

In [103]:
"""
#random forest
reg = RandomForestRegressor(n_estimators=100, random_state=42)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("Random Forest Regression MSE:", mse)
print("Random Forest Regression RMSE:", rmse)
"""

'\n#random forest\nreg = RandomForestRegressor(n_estimators=100, random_state=42)\nreg.fit(X_train, y_train)\ny_pred = reg.predict(X_test)\n\nmse = mean_squared_error(y_test, y_pred)\nrmse = np.sqrt(mse)\nprint("Random Forest Regression MSE:", mse)\nprint("Random Forest Regression RMSE:", rmse)\n'

In [104]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

#treat target as if it were categorical ints rather than continuous float
#this becomes a problem of classification rather than regression
#use models that allow for using class weights to counteract the imbalance that arises when most collisions have 0 people killed

df['NUMBER OF PERSONS KILLED'] = df["NUMBER OF PERSONS KILLED"].astype(int)

X = df.drop(columns=['NUMBER OF PERSONS KILLED'])
y = df['NUMBER OF PERSONS KILLED']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [105]:
#decision tree with 1 level for baseline oneR
classifier = DecisionTreeClassifier(class_weight='balanced', random_state=42, max_depth=1)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

cm = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix:\n{cm}")

Accuracy: 5.139518150406022e-05
Confusion Matrix:
[[     0      0 252003      0    561      0]
 [     0      0    356      0      2      0]
 [     0      0     13      0      0      0]
 [     0      0      4      0      0      0]
 [     0      0      2      0      0      0]
 [     0      0      1      0      0      0]]


In [106]:
#logistic regression
reg = LogisticRegression(class_weight='balanced', max_iter=1000)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

cm = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix:\n{cm}")

Accuracy: 0.5516007622300765
Confusion Matrix:
[[139414  45327  57041  10770     12      0]
 [   150    103     81     23      1      0]
 [     5      1      6      1      0      0]
 [     0      3      1      0      0      0]
 [     1      1      0      0      0      0]
 [     1      0      0      0      0      0]]


In [107]:
#decision tree
classifier = DecisionTreeClassifier(class_weight='balanced', random_state=42)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

cm = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix:\n{cm}")

Accuracy: 0.997050707276767
Confusion Matrix:
[[252193    360      8      3      0      0]
 [   355      3      0      0      0      0]
 [    13      0      0      0      0      0]
 [     4      0      0      0      0      0]
 [     2      0      0      0      0      0]
 [     1      0      0      0      0      0]]


In [108]:
#random forest
classifier = RandomForestClassifier(class_weight='balanced', n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

cm = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix:\n{cm}")

Accuracy: 0.9984976793098813
Confusion Matrix:
[[252562      2      0      0      0      0]
 [   358      0      0      0      0      0]
 [    13      0      0      0      0      0]
 [     4      0      0      0      0      0]
 [     2      0      0      0      0      0]
 [     1      0      0      0      0      0]]


In [109]:
unique_death_counts = df['NUMBER OF PERSONS KILLED'].value_counts()
print(unique_death_counts)

NUMBER OF PERSONS KILLED
0    1262772
1       1877
2         47
3          9
4          3
8          1
Name: count, dtype: int64
